As importações das bibliotecas e ferramentas que iremos usar.

In [1]:
from enum import auto, IntFlag
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError,
)

Criação da classe Role (Papel/função) herdada de IntFlag, que vai ser um enumerador, ou seja, irá servir para definir os papéis que o usuário irá possuir, utilizado de bits com identificação, 1 (001), 2 (010), 4 (100) e 7 (111).

In [2]:
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer

Define a classe que iremos trabalhar durante todos os exemplos, usando a ferramenta "Field" que dá a possibilidade de colocar mais informações sobre esse campo, colocar regras e estados. Possuindo 4 atributo: "name" do tipo string, com um exemplo; "email" do tipo EmailStr(string padrão de email), com um exemplo, uma descrição e sem possibilidade de mudar (frozen=True); "password" do tipo SecretStr(string criptografada), com exemplo e descrição; e por último "role" do tipo Role (classe que acabamos de criar), sendo o padrão, se não informado, vazio.

In [3]:

class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["example@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(default=None, description="The role of the user")


A função abaixo é uma das principais ferramentas que a Pydantic oferece, de maneira que irá ter o papel de validar se um input em um objeto é válido ou não. Possuindo, como entrada, um dicionário (ainda não é o objeto porque estamos fazendo a checagem se a entrada é válida para, só então colocar na instância). Então, o método "model_validate" é acionado, analisando se todos os campos do modelo satisfazem as condições impostas, se sim é imprimido o usuário, se não é imprimido o erro.

In [4]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid")
        for error in e.errors():
            print(error)

Apenas dá um exemplo de um dado "bom" e um "ruim", invocando a função validate nos dois casos. No primeiro temos a informação dos usuários, ou seja, são dados válidos. Porém, no segundo exemplo, é possível ver que teve erros do tipo "missing" e "value_error" nos campos de "name" e "email" respectivamente, uma descrição desse erro, e os dados.

In [5]:
def main() -> None:
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
    bad_data = {"email": "<bad data>", "password": "<bad data>"}

    validate(good_data)
    validate(bad_data)


if __name__ == "__main__":
    main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.12/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}
